In [106]:
from langchain_openai import ChatOpenAI

chatModel = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [107]:
from langchain.chains import LLMChain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a curious fact about {soccer_player}")

traditional_chain = LLMChain(
    llm=chatModel,
    prompt=prompt
)

In [108]:
traditional_chain.predict(soccer_player="Maradona")

'One curious fact about Maradona is that he has his own religion called the "Iglesia Maradoniana" (Church of Maradona) in Argentina, where he is worshipped as a god-like figure by his fans. The church has its own "Ten Commandments" based on Maradona\'s life and career.'

In [109]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [ ]:

prompt = ChatPromptTemplate.from_template("tell me a curious fact about {crickter}")

chain = prompt | chatModel | StrOutputParser()

# traditional_chain = LLMChain(
#     llm=chatModel,
#     prompt=prompt
# )

In [112]:
chain.invoke({"crickter": "rohitsharma"})


'Rohit Sharma is the only cricketer in the world to have scored three double centuries in One Day Internationals (ODIs).'

# LCEL 

## RunnablePassthrough
* It does not do anything to the input data.
* Let's see it in a very simple example: a chain with just RunnablePassthrough() will output the original input without any modification.

In [114]:

from langchain_core.runnables import RunnablePassthrough
chain=RunnablePassthrough()

In [115]:
chain.invoke("prince katiyar")

'prince katiyar'

In [116]:
chain.invoke("this is a live xlass")

'this is a live xlass'

## RunnableLambda
* To use a custom function inside a LCEL chain we need to wrap it up with RunnableLambda.
* Let's define a very simple function to create Russian lastnames:

In [117]:
from langchain_core.runnables import RunnableLambda

In [118]:
def russian_lastname(name: str) -> str:
    return f"{name}ovich"

In [119]:

chain = RunnablePassthrough() | RunnableLambda(russian_lastname)

In [121]:
chain.invoke("prince katiyar")

'prince katiyarovich'

## RunnableParallel
* We will use RunnableParallel() for running tasks in parallel.
* This is probably the most important and most useful Runnable from LangChain.
* In the following chain, RunnableParallel is going to run these two tasks in parallel:
    * operation_a will use RunnablePassthrough.
    * operation_b will use RunnableLambda with the russian_lastname function.

In [122]:
from langchain_core.runnables import RunnableParallel

In [123]:
chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "operation_b": RunnableLambda(russian_lastname)
    }
)

In [124]:
chain.invoke("prince")

{'operation_a': 'prince', 'operation_b': 'princeovich'}

In [131]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [132]:
prompt = ChatPromptTemplate.from_template("tell me a curious fact about {soccer_player}")

output_parser = StrOutputParser()


In [133]:
def russian_lastname_from_dictionary(person):
    return person["name"] + "ovich"

In [134]:
chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "soccer_player": RunnableLambda(russian_lastname_from_dictionary),
        "operation_c": RunnablePassthrough(),
    }

    
) | prompt | model | output_parser

In [135]:
chain.invoke({
    "name1": "Jordam",
     "name": "Abram"})

"One curious fact about Roman Abramovich is that he is known for his extravagant spending habits, including owning multiple luxury properties, yachts, and a private jet. He also famously purchased the world's most expensive yacht, the Eclipse, for over $500 million. Additionally, Abramovich is known for his generous philanthropy, having donated millions of dollars to various charitable causes and organizations."

# Built in Fucntion in Runnables : BIND

In [158]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [159]:
prompt = ChatPromptTemplate.from_template("tell me a achivemet and award  fact about {soccer_player}")

output_parser = StrOutputParser()

In [160]:
chain = prompt | model | output_parser

In [161]:

chain.invoke({"soccer_player": "Ronaldo"})

"One of Cristiano Ronaldo's notable achievements is winning the prestigious FIFA Ballon d'Or award five times, making him one of the most decorated footballers in history."

In [ ]:
chain = prompt | model.bind(stop=["Ronaldo"]) | output_parser

# chain = prompt | model | output_parser

## Use of .bind() to add arguments to a Runnable in a LCEL Chain
* For example, we can add an argument to stop the model response when it reaches the word "Ronaldo":

# Combine chain


In [136]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")

In [137]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a sentence about {politician}")

chain = prompt | model | StrOutputParser()

In [138]:
chain.invoke("modi")

'Narendra Modi is the current Prime Minister of India.'

In [142]:
historian_prompt = ChatPromptTemplate.from_template("Was {mychain} positive for Humanity?")

In [143]:
composed_chain = {"mychain": chain} | historian_prompt | model | StrOutputParser()

# chain = prompt | model | StrOutputParser()

In [145]:
composed_chain.invoke({"politician": "modi"})

"Opinions on Narendra Modi's impact on humanity are deeply divided. Supporters argue that his policies have led to economic growth and development in India, benefiting millions of people. They also point to his efforts to improve sanitation, healthcare, and education in the country.\n\nOn the other hand, critics point to allegations of human rights abuses, including the treatment of religious minorities and marginalized communities, as well as concerns about freedom of speech and press. Modi's handling of the COVID-19 pandemic has also been criticized, with some accusing him of failing to adequately protect the population.\n\nUltimately, whether Narendra Modi's leadership has been positive for humanity is a complex and contentious issue that depends on individual perspectives and priorities."

In [ ]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [ ]:
prompt1 = ChatPromptTemplate.from_template("what is the country {politician} is from?")

prompt2 = ChatPromptTemplate.from_template(
    "what continent is the country {country} in? respond in {language}"
)

model = ChatOpenAI()

chain1 = prompt1 | model | StrOutputParser()

chain2 = (
    {"country": chain1, }
    | prompt2
    | model
    | StrOutputParser()
)

In [ ]:
prompt1 = ChatPromptTemplate.from_template("what is the country {politician} is from?")

model = ChatOpenAI()


chain1 = prompt1 | model | StrOutputParser()
chain1.invoke({"politician": "modi"})

'Narendra Modi is from India.'

In [154]:
prompt2 = ChatPromptTemplate.from_template(
    "what continent is the country {country} in? respond in {language}"
)

chain2 = (
    {"country": chain1, "language": itemgetter("language")}
    | prompt2
    | model
    | StrOutputParser()
)

In [157]:
chain2.invoke({"politician": "trump", "language": "bhajouri"})

'Donald Trump के देश संयुक्त राज्य अमेरिका है जो उत्तरी अमेरिका में है।'

# RAG 